In [47]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

# Note: The following do not work with Python 3.12
#import sweetviz as sv
#import shap
#from ydata_profiling import ProfileReport

In [48]:
seed = 2024

# pandas, statsmodels, matplotlib and y_data_profiling rely on numpy's random generator, and thus, we need to set the seed in numpy
np.random.seed(seed)

Add diet category to the recipes.csv dataset.

In [49]:
# Read the CSV file
df = pd.read_csv('data/recipes.csv')

# drop all tables except RecipeId, RecipeIngredientParts
df = df[['RecipeId', 'RecipeIngredientParts']]
df.head()
df.info()

# iterate over all RecipeIngredientParts and extract the ingredients uniquely
ingredients = set()
for row in df.itertuples():
    for ingredient in row.RecipeIngredientParts.split(','):
        ingredients.add(ingredient.strip())

# remove all c("\", "(", ")") and c"" and ""
ingredients = [ingredient.replace('\\', '').replace('(', '').replace(')', '').replace('c""', '').replace('""', '').replace('"', '').strip() for ingredient in ingredients]

# now merge all ingredients into one string
ingredients_string = ','.join(ingredients)
print(ingredients_string)

# replace all , with " " and split the string into a list
ingredients_list = ingredients_string.replace(',', ' ').split(' ')

print(ingredients_list)

# create a set to remove all duplicates
unique_ingredients = set(ingredients_list)
print(unique_ingredients)
print(len(unique_ingredients))

# export it into a text file
with open('data/unique_ingredients.txt', 'w') as f:
    for item in unique_ingredients:
        f.write("%s\n" % item)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75604 entries, 0 to 75603
Data columns (total 2 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   RecipeId               75604 non-null  int64 
 1   RecipeIngredientParts  75604 non-null  object
dtypes: int64(1), object(1)
memory usage: 1.2+ MB
lebanese cucumbers,salted cashews,granola cereal,kalamata olive,sweetened condensed milk,sausage meat,beefsteak tomato,berry-flavored vodka,tofu crumbles,green cabbage,firm silken tofu,chocolate soymilk,portabella mushrooms,button mushroom,boneless beef brisket,low joule gelatin,portabella mushroom caps,vital wheat gluten,baby shrimp,mozzarella-cheddar blend cheese,fresh spinach leaves,2% evaporated milk,nacho cheese flavored tortilla chips,mandarin orange,serrano chilies,almond paste,strawberry syrup,nectarine,fresh sage leaf,cannellini beans,chicken flavor instant bouillon,unsweetened applesauce,oats,basil leaves,plain gelatin,l

In [50]:
# Read the CSV file
df = pd.read_csv('data/recipes.csv')

# Consolidated Non-Vegetarian Keywords
non_vegetarian_keywords = list(set([
    'flounder', 'lobsters', 'lump', 'rack', 'shank', 'steak', 'scallops', 'alligator', 
    'livers', 'roe', 'ham', 'turkey', 'chicken', 'duck', 'bacon', 'tuna', 'swordfish', 
    'lobster', 'meatballs', 'salmon', 'sweetbreads', 'breasts', 'chicken-flavored', 
    'ducklings', 'drumstick', 'liver', 'shanks', 'rabbit', 'poultry', 'herring', 
    'mussels', 'clams', 'squid', 'pork', 'veal', 'haddock', 'chorizo', 'chihuahua', 
    'eel', 'stuffing', 'cod', 'gelatin', 'sausage', 'curd', 'thighs', 'lox', 'cabbage', 
    'wonton', 'bone', 'giblets', 'pheasant', 'quail', 'shrimp', 'fish', 'sole', 
    'gizzard', 'Canadian', 'pesto', 'truffles', 'anchovies', 'venison', 'pheasants', 
    'tenderloin', 'meats', 'tripe', 'breast', 'wings', 'ribs', 'sausages', 'trout', 
    'oysters', 'octopus', 'crab', 'prawns', 'catfish', 'sardines', 'mahi', 'halibut', 
    'bass', 'perch', 'tilapia', 'grouper'
]))

# Consolidated Non-Vegan Keywords
non_vegan_keywords = list(set([
    'milk', 'cheese', 'butter', 'egg', 'honey', 'mozzarella-cheddar', 'cream', 'whip', 
    'jarlsberg', 'fontina', 'ham', 'cheesecake', 'hollandaise', 'caviar', 'creamRegular', 
    'custard', 'yogurt', 'gouda', 'margarine', 'beef', 'salmon', 'sour', 'bisquick', 
    'carton', 'cotija', 'creme', 'buttercream', 'buttermilk', 'ricotta', 'cottage', 
    'eggs', 'mayonnaise', 'eggshells', 'lactose-free', 'skim', 'ghee', 'mascarpone', 
    'alfredo', 'whey', 'casein', 'lactose', 'albumin', 'bechamel', 'sour cream', 
    'cream cheese', 'feta', 'gorgonzola', 'parmesan', 'mozzarella', 'cheddar', 'brie', 
    'camembert', 'roquefort', 'stilton', 'blue cheese', 'colby', 'monterey jack', 
    'swiss cheese', 'provolone', 'edam', 'havarti', 'pecorino', 'asiago', 'emmental', 
    'gruyere', 'halloumi', 'manchego', 'paneer', 'queso fresco', 'ricotta salata', 
    'romano', 'taleggio', 'vacherin', 'milk chocolate', 'whey protein', 'casein protein', 
    'egg noodles', 'egg whites', 'egg yolks', 'hollandaise sauce', 'aioli', 'flan', 
    'quiche', 'meringue', 'pavlova', 'egg wash', 'frittata', 'omelette', 'scrambled eggs', 
    'poached eggs', 'hard-boiled eggs', 'deviled eggs', 'eggnog', 'brioche', 'challah', 
    'pound cake', 'sponge cake', 'angel food cake', 'ladyfingers', 'mousse', 'souffle', 
    'creme brulee', 'panna cotta', 'tiramisu', 'yorkshire pudding', 'beef broth', 
    'chicken broth', 'fish sauce', 'oyster sauce', 'worcestershire sauce', 'caesar dressing', 
    'carbonara sauce', 'béarnaise sauce', 'gravlax', 'smoked salmon', 'caviar', 'anchovy paste', 
    'fish stock'
]))


# Function to check if a RecipeIngredientParts is vegetarian
def is_vegetarian(ingredient):
    for keyword in non_vegetarian_keywords:
        if keyword in ingredient.lower():
            return False
    return True

# Function to check if a RecipeIngredientParts is vegan
def is_vegan(ingredient):
    for keyword in non_vegan_keywords:
        if keyword in ingredient.lower():
            return False
    return True

# Apply the is_vegetarian function to the RecipeIngredientParts column
df['is_vegetarian'] = df['RecipeIngredientParts'].apply(is_vegetarian)

# Apply the is_vegan function to the RecipeIngredientParts column
df['is_vegan'] = df['RecipeIngredientParts'].apply(is_vegan)

# Map the diet category based on the is_vegetarian and is_vegan columns
df['diet_category'] = df.apply(lambda row: 'Vegetarian' if row['is_vegetarian'] else 'Vegan' if row['is_vegan'] else 'Omnivore', axis=1)

df.head()

# remove is_vegetarian and is_vegan columns
df = df.drop(columns=['is_vegetarian', 'is_vegan'])

# Print the dataframe with the added is_vegetarian, is_vegan, and diet_category columns
print(df)

# change the df name to df_recipes
df_recipes = df

       RecipeId                                            Name  CookTime  \
0         73440                    Bow Ties With Broccoli Pesto         0   
1        365718                             Cashew-chutney Rice      3600   
2        141757        Copycat Taco Bell Nacho Fries BellGrande      3600   
3        280351        Slow Cooker Jalapeno Cheddar Cheese Soup     18000   
4        180505                 Cool & Crisp Citrus Chiffon Pie      3600   
...         ...                                             ...       ...   
75599    253577  Frijoles Negros- Crock Pot Mexican Black Beans     43200   
75600    267827                                  Moose Moussaka      3600   
75601    266983        Cantonese Pepper Steak for Two (Or More)      1800   
75602    253739                            Coconut Cream Cooler       300   
75603     78171                                 Cheater Risotto       960   

       PrepTime RecipeCategory  \
0          1800          Other   
1      

Readin data and preprocessing

In [51]:
# readin the diet data
df_diet = pd.read_csv('data/diet.csv')

# chcek for missing values in the data
print(df_diet.isnull().sum())

# replace missing value in Diet with "Omnivore"
print(df_diet["Diet"].unique())
df_diet["Diet"] = df_diet["Diet"].fillna("Omnivore")

# check again
print(df_diet.isnull().sum())

# Change data type of Diet to category
df_diet["Diet"] = df_diet["Diet"].astype("category")

df_diet.info()


AuthorId    0
Diet        1
Age         0
dtype: int64
['Vegetarian' 'Vegan' 'Omnivore' nan]
AuthorId    0
Diet        0
Age         0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271907 entries, 0 to 271906
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype   
---  ------    --------------   -----   
 0   AuthorId  271907 non-null  object  
 1   Diet      271907 non-null  category
 2   Age       271907 non-null  int64   
dtypes: category(1), int64(1), object(1)
memory usage: 4.4+ MB


In [52]:
# read csv file
df_requests = pd.read_csv('data/requests.csv')

# check for missing values
print(df_requests.isnull().sum())

# HighCalories is a 1.0 and 0.0, change to: true for it should have high calories and false for low calories.
# Type bool is not able to represent NA -> type boolean is needed after merging

df_requests['HighCalories'] = df_requests['HighCalories'].astype('boolean')

# HighProtein is Indifferent and Yes, change to: true for it should have high protein and false for low protein.

df_requests['HighProtein'] = df_requests['HighProtein'].replace({'Indifferent': False, 'Yes': True})
df_requests['HighProtein'] = df_requests['HighProtein'].astype('boolean')

# LowFat is a 1 and 0, change to: true for it should have low fat and false for high fat.

df_requests['LowFat'] = df_requests['LowFat'].astype('boolean')

# LowSugar is Indifferent and 0, change to: true for it should have low sugar and false for high sugar.

df_requests['LowSugar'] = df_requests['LowSugar'].replace({'Indifferent': False, '0': True})
df_requests['LowSugar'] = df_requests['LowSugar'].astype('boolean')

# HighFiber is 1 and 0, change to: true for it should have high fiber and false for low fiber.

df_requests['HighFiber'] = df_requests['HighFiber'].astype('boolean')

# print(df_requests.head(8))
print(df_requests.info())


AuthorId        0
RecipeId        0
Time            0
HighCalories    0
HighProtein     0
LowFat          0
LowSugar        0
HighFiber       0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140195 entries, 0 to 140194
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   AuthorId      140195 non-null  object 
 1   RecipeId      140195 non-null  int64  
 2   Time          140195 non-null  float64
 3   HighCalories  140195 non-null  boolean
 4   HighProtein   140195 non-null  boolean
 5   LowFat        140195 non-null  boolean
 6   LowSugar      140195 non-null  boolean
 7   HighFiber     140195 non-null  boolean
dtypes: boolean(5), float64(1), int64(1), object(1)
memory usage: 4.5+ MB
None


In [53]:
merged_df = df_diet.merge(df_requests, on='AuthorId', how='left')

print(merged_df.info())

print(merged_df.head(8))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 362754 entries, 0 to 362753
Data columns (total 10 columns):
 #   Column        Non-Null Count   Dtype   
---  ------        --------------   -----   
 0   AuthorId      362754 non-null  object  
 1   Diet          362754 non-null  category
 2   Age           362754 non-null  int64   
 3   RecipeId      140195 non-null  float64 
 4   Time          140195 non-null  float64 
 5   HighCalories  140195 non-null  boolean 
 6   HighProtein   140195 non-null  boolean 
 7   LowFat        140195 non-null  boolean 
 8   LowSugar      140195 non-null  boolean 
 9   HighFiber     140195 non-null  boolean 
dtypes: boolean(5), category(1), float64(2), int64(1), object(1)
memory usage: 14.9+ MB
None
    AuthorId        Diet  Age  RecipeId       Time  HighCalories  HighProtein  \
0  10000120E  Vegetarian   46       NaN        NaN          <NA>         <NA>   
1   1000014D       Vegan   18       NaN        NaN          <NA>         <NA>   
2   1000015A 

Now clean recipes and merge it

In [54]:
#load the data
# df_recipes = pd.read_csv('data/recipes.csv')

#check for missing values
print(df_recipes.isnull().sum())

#drop the missing values
# df_recipes = df_recipes.dropna()

df_recipes["RecipeCategory"] = df_recipes["RecipeCategory"].astype("category")
df_recipes["diet_category"] = df_recipes["diet_category"].astype("category")

RecipeId                          0
Name                              0
CookTime                          0
PrepTime                          0
RecipeCategory                    0
RecipeIngredientQuantities        0
RecipeIngredientParts             0
Calories                          0
FatContent                        0
SaturatedFatContent               0
CholesterolContent                0
SodiumContent                     0
CarbohydrateContent               0
FiberContent                      0
SugarContent                      0
ProteinContent                    0
RecipeServings                26713
RecipeYield                   50295
diet_category                     0
dtype: int64


In [55]:
merged_df = merged_df.merge(df_recipes, on='RecipeId', how='left')

print(merged_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 362754 entries, 0 to 362753
Data columns (total 28 columns):
 #   Column                      Non-Null Count   Dtype   
---  ------                      --------------   -----   
 0   AuthorId                    362754 non-null  object  
 1   Diet                        362754 non-null  category
 2   Age                         362754 non-null  int64   
 3   RecipeId                    140195 non-null  float64 
 4   Time                        140195 non-null  float64 
 5   HighCalories                140195 non-null  boolean 
 6   HighProtein                 140195 non-null  boolean 
 7   LowFat                      140195 non-null  boolean 
 8   LowSugar                    140195 non-null  boolean 
 9   HighFiber                   140195 non-null  boolean 
 10  Name                        140195 non-null  object  
 11  CookTime                    140195 non-null  float64 
 12  PrepTime                    140195 non-null  float64 
 13 

In [56]:
# load reviews data
df_reviews = pd.read_csv('data/reviews.csv')

print(df_reviews.info())

# df_reviews['Rating'] = df_reviews['Rating'].fillna(0)
# sns.countplot(data=df_reviews, x='Rating')

# drop Rating column
df_reviews = df_reviews.drop('Rating', axis=1)

# drop all rows having a TestSetId
df_reviews = df_reviews[df_reviews['TestSetId'].isna()]

# drop TestSetId column
df_reviews = df_reviews.drop('TestSetId', axis=1)

# check for missing values
# print(df_reviews.isnull().sum())

# change the data type of Like to boolean
df_reviews['Like'] = df_reviews['Like'].astype('boolean')

print(df_reviews.info())
print(df_reviews.head(8))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140195 entries, 0 to 140194
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   AuthorId   140195 non-null  object 
 1   RecipeId   140195 non-null  int64  
 2   Rating     77108 non-null   float64
 3   Like       97381 non-null   object 
 4   TestSetId  42814 non-null   float64
dtypes: float64(2), int64(1), object(2)
memory usage: 5.3+ MB
None
<class 'pandas.core.frame.DataFrame'>
Index: 97381 entries, 42814 to 140194
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   AuthorId  97381 non-null  object 
 1   RecipeId  97381 non-null  int64  
 2   Like      97381 non-null  boolean
dtypes: boolean(1), int64(1), object(1)
memory usage: 2.4+ MB
None
       AuthorId  RecipeId   Like
42814  1000036C    320576  False
42815  1000216B    189335  False
42816  1000221A    133043  False
42817  1000221A     90537  False
4

/var/folders/9c/9ttnz44d25g9j7t3x_fdr1d40000gn/T/ipykernel_81749/1761953307.py:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df_reviews = pd.read_csv('data/reviews.csv')


In [57]:
# now merge the reviews with the merged_df on the RecipeId and AuthorId
merged_df = merged_df.merge(df_reviews, on=['RecipeId', 'AuthorId'], how='left')

print(merged_df.info())
print(merged_df.head(8))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 362754 entries, 0 to 362753
Data columns (total 29 columns):
 #   Column                      Non-Null Count   Dtype   
---  ------                      --------------   -----   
 0   AuthorId                    362754 non-null  object  
 1   Diet                        362754 non-null  category
 2   Age                         362754 non-null  int64   
 3   RecipeId                    140195 non-null  float64 
 4   Time                        140195 non-null  float64 
 5   HighCalories                140195 non-null  boolean 
 6   HighProtein                 140195 non-null  boolean 
 7   LowFat                      140195 non-null  boolean 
 8   LowSugar                    140195 non-null  boolean 
 9   HighFiber                   140195 non-null  boolean 
 10  Name                        140195 non-null  object  
 11  CookTime                    140195 non-null  float64 
 12  PrepTime                    140195 non-null  float64 
 13 

Now everthing is merged into one dataframe and ready to be used for analysis!!!

In [58]:
# Add PrepTime and CookTime to TotalTime

merged_df['TotalTime'] = (merged_df['PrepTime'] + merged_df['CookTime']).astype(float)

#Check
print(merged_df[['PrepTime','CookTime','TotalTime']].head(10))
print(merged_df.info())

# drop columns PrepTime and CookTime
merged_df = merged_df.drop('PrepTime', axis=1)
merged_df = merged_df.drop('CookTime', axis=1)

print(merged_df.info())

   PrepTime  CookTime  TotalTime
0       NaN       NaN        NaN
1       NaN       NaN        NaN
2       NaN       NaN        NaN
3       NaN       NaN        NaN
4       NaN       NaN        NaN
5       NaN       NaN        NaN
6     120.0       0.0      120.0
7       NaN       NaN        NaN
8       NaN       NaN        NaN
9       NaN       NaN        NaN
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 362754 entries, 0 to 362753
Data columns (total 30 columns):
 #   Column                      Non-Null Count   Dtype   
---  ------                      --------------   -----   
 0   AuthorId                    362754 non-null  object  
 1   Diet                        362754 non-null  category
 2   Age                         362754 non-null  int64   
 3   RecipeId                    140195 non-null  float64 
 4   Time                        140195 non-null  float64 
 5   HighCalories                140195 non-null  boolean 
 6   HighProtein                 140195 non-null  bool

In [59]:
# Drop all row having no Like value
merged_df = merged_df[merged_df['Like'].notna()]

# drop unncecessary columns, RecipeServings, RecipeYield, AuthorId, RecipeID, Name? (maybe not), RecipeIngredientParts, RecipeIngredientQuantities
merged_df = merged_df.drop('RecipeServings', axis=1)
merged_df = merged_df.drop('RecipeYield', axis=1)
merged_df = merged_df.drop('AuthorId', axis=1)
merged_df = merged_df.drop('RecipeId', axis=1)
merged_df = merged_df.drop('Name', axis=1)
merged_df = merged_df.drop('RecipeIngredientParts', axis=1)
merged_df = merged_df.drop('RecipeIngredientQuantities', axis=1)

# check again if datatypes is correct -> e.g. Like should be boolean
# -> chnaged data type from bool to boolean, bc bool can not hold NAs, whcih is necessary after merging

# WJ Jacqueline will do this part (upper)

# extract outlier, e.g. <4h drop values -> logical threshold (everyone by themselves)


# time definition should be smaller than the recipe time (Threshhold 10~20min maybe)
# biniry pca Or clustering of time -> Han-Gyu

# we want to create binary classification of Calories, Protein, Fat, Sugar, Fiber, and Sodium
# True for high and False for low
# first we need to create a threshhold for each of the columns -> Ngoc

# spätestens Sonntag 10am.

In [60]:
print(merged_df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 97381 entries, 6 to 362749
Data columns (total 21 columns):
 #   Column               Non-Null Count  Dtype   
---  ------               --------------  -----   
 0   Diet                 97381 non-null  category
 1   Age                  97381 non-null  int64   
 2   Time                 97381 non-null  float64 
 3   HighCalories         97381 non-null  boolean 
 4   HighProtein          97381 non-null  boolean 
 5   LowFat               97381 non-null  boolean 
 6   LowSugar             97381 non-null  boolean 
 7   HighFiber            97381 non-null  boolean 
 8   RecipeCategory       97381 non-null  category
 9   Calories             97381 non-null  float64 
 10  FatContent           97381 non-null  float64 
 11  SaturatedFatContent  97381 non-null  float64 
 12  CholesterolContent   97381 non-null  float64 
 13  SodiumContent        97381 non-null  float64 
 14  CarbohydrateContent  97381 non-null  float64 
 15  FiberContent         97